In [28]:
 # Import modules
from warnings import simplefilter
simplefilter(action='ignore', category=FutureWarning)
import pandas as pd
from pathlib import Path
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.preprocessing import StandardScaler

import sqlite3

In [29]:
conn = sqlite3.connect('nfl_data.sqlite')
df = pd.read_sql('SELECT * FROM relative_team_formations', conn).set_index('play_uuid').dropna()

# Review the DataFrame
df.head()

,off_x0,off_y0,off_x1,off_y1,off_x2,off_y2,off_x3,off_y3,off_x4,off_y4,...,yardsToGo,preSnapHomeScore,preSnapVisitorScore,offenseFormation,defendersInTheBox,preSnapHomeTeamWinProbability,preSnapVisitorTeamWinProbability,homeTeamWinProbabilityAdded,visitorTeamWinProbilityAdded,down
play_uuid,,,,,,,,,,,,,,,,,,,,,
2022090800.101,-1.089999,-3.890000,-0.459999,-5.690000,-0.989999,-15.720000,-1.109999,-2.270000,-1.509999,-5.780000,...,10,0,0,I_FORM,6.0,0.399819,0.600181,-0.014850,0.014850,1
2022090800.1187,-1.650000,-5.850001,-0.660000,-19.840001,-1.150000,-8.990001,-1.540000,-1.530001,-0.750000,-0.140001,...,10,0,10,SINGLEBACK,6.0,0.207237,0.792763,0.006345,-0.006345,1
2022090800.1254,-5.120001,5.790001,-0.910001,20.080001,-1.390001,2.780001,-0.730001,-15.669999,-1.570001,-6.229999,...,10,0,10,PISTOL,6.0,0.227212,0.772788,0.008701,-0.008701,1
2022090800.1385,-1.649998,-5.930000,-0.579998,-13.410000,-1.219998,-8.780000,-1.029998,-1.670000,-1.649998,-10.800000,...,10,0,10,SINGLEBACK,6.0,0.235201,0.764799,0.027923,-0.027923,1
2022090800.1406,-1.260001,6.610000,-0.280001,20.430000,-0.530001,3.470000,-1.020001,1.580000,-0.480001,0.450000,...,7,0,10,SINGLEBACK,6.0,0.263124,0.736876,0.000100,-0.000100,1


In [30]:
 # Seperate the features, X,  from the target variable, y
y = df['converted']
X = df.drop(columns=['converted'])

In [31]:
X

,off_x0,off_y0,off_x1,off_y1,off_x2,off_y2,off_x3,off_y3,off_x4,off_y4,...,yardsToGo,preSnapHomeScore,preSnapVisitorScore,offenseFormation,defendersInTheBox,preSnapHomeTeamWinProbability,preSnapVisitorTeamWinProbability,homeTeamWinProbabilityAdded,visitorTeamWinProbilityAdded,down
play_uuid,,,,,,,,,,,,,,,,,,,,,
2022090800.101,-1.089999,-3.890000,-0.459999,-5.690000,-0.989999,-15.720000,-1.109999,-2.270000,-1.509999,-5.780000,...,10,0,0,I_FORM,6.0,0.399819,0.600181,-0.014850,0.014850,1
2022090800.1187,-1.650000,-5.850001,-0.660000,-19.840001,-1.150000,-8.990001,-1.540000,-1.530001,-0.750000,-0.140001,...,10,0,10,SINGLEBACK,6.0,0.207237,0.792763,0.006345,-0.006345,1
2022090800.1254,-5.120001,5.790001,-0.910001,20.080001,-1.390001,2.780001,-0.730001,-15.669999,-1.570001,-6.229999,...,10,0,10,PISTOL,6.0,0.227212,0.772788,0.008701,-0.008701,1
2022090800.1385,-1.649998,-5.930000,-0.579998,-13.410000,-1.219998,-8.780000,-1.029998,-1.670000,-1.649998,-10.800000,...,10,0,10,SINGLEBACK,6.0,0.235201,0.764799,0.027923,-0.027923,1
2022090800.1406,-1.260001,6.610000,-0.280001,20.430000,-0.530001,3.470000,-1.020001,1.580000,-0.480001,0.450000,...,7,0,10,SINGLEBACK,6.0,0.263124,0.736876,0.000100,-0.000100,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022110700.612,-0.930002,-1.250000,-1.260002,-2.790000,-1.310002,3.480000,-0.890002,-18.910000,-2.860002,3.890000,...,10,0,0,PISTOL,6.0,0.466372,0.533628,0.047439,-0.047439,1
2022110700.670,-0.979999,-1.369999,-1.169999,-2.789999,-1.079999,3.270001,-7.299999,0.180001,-4.349999,-2.019999,...,1,0,0,PISTOL,7.0,0.459646,0.540354,-0.023838,0.023838,3
2022110700.80,-4.479998,-6.710000,-0.779998,-3.400000,-0.569998,-5.160000,-0.669998,-9.790000,-5.289998,-4.270000,...,10,0,0,SHOTGUN,7.0,0.583861,0.416139,-0.001371,0.001371,1


In [32]:
X = pd.get_dummies(X)
X.head()

,off_x0,off_y0,off_x1,off_y1,off_x2,off_y2,off_x3,off_y3,off_x4,off_y4,...,ballDirection_middle,ballDirection_none,ballDirection_right,offenseFormation_EMPTY,offenseFormation_I_FORM,offenseFormation_JUMBO,offenseFormation_PISTOL,offenseFormation_SHOTGUN,offenseFormation_SINGLEBACK,offenseFormation_WILDCAT
play_uuid,,,,,,,,,,,,,,,,,,,,,
2022090800.101,-1.089999,-3.890000,-0.459999,-5.690000,-0.989999,-15.720000,-1.109999,-2.270000,-1.509999,-5.780000,...,False,False,True,False,True,False,False,False,False,False
2022090800.1187,-1.650000,-5.850001,-0.660000,-19.840001,-1.150000,-8.990001,-1.540000,-1.530001,-0.750000,-0.140001,...,False,False,True,False,False,False,False,False,True,False
2022090800.1254,-5.120001,5.790001,-0.910001,20.080001,-1.390001,2.780001,-0.730001,-15.669999,-1.570001,-6.229999,...,False,False,False,False,False,False,True,False,False,False
2022090800.1385,-1.649998,-5.930000,-0.579998,-13.410000,-1.219998,-8.780000,-1.029998,-1.670000,-1.649998,-10.800000,...,False,False,False,False,False,False,False,False,True,False
2022090800.1406,-1.260001,6.610000,-0.280001,20.430000,-0.530001,3.470000,-1.020001,1.580000,-0.480001,0.450000,...,True,False,False,False,False,False,False,False,True,False


In [33]:
# Split the dataset using train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

In [34]:
 # Instantiate a StandardScaler instance
scaler = StandardScaler()

# Fit the training data to the standard scaler
X_scaler = scaler.fit(X_train)

# Transform the training data using the scaler
X_train_scaled = X_scaler.transform(X_train)

# Transform the testing data using the scaler
X_test_scaled = X_scaler.transform(X_test)

In [35]:
# Import the KNeighborsClassifier module from sklearn
from sklearn.neighbors import KNeighborsClassifier

# Instantiate the KNeighborsClassifier model with n_neighbors = 3
knn = KNeighborsClassifier(n_neighbors=10)

In [36]:
# Train the model using the training data
knn.fit(X_train_scaled, y_train)

KNeighborsClassifier(n_neighbors=10)

In [37]:
# Create predictions using the testing data
y_pred = knn.predict(X_test_scaled)

In [38]:
print(confusion_matrix(y_test, y_pred))

[[1168   18]
 [ 343   46]]


In [39]:
 # Print the classification report comparing the testing data to the model predictions
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.77      0.98      0.87      1186
           1       0.72      0.12      0.20       389

    accuracy                           0.77      1575
   macro avg       0.75      0.55      0.53      1575
weighted avg       0.76      0.77      0.70      1575

